In [25]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [26]:
dataset=pd.read_csv("CKD.csv")

In [27]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [28]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [29]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [30]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [31]:
indep

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes
0,2.000000,76.459948,3,0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,1,0,0,0,0,0,0,1,1,0
1,3.000000,76.459948,2,0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,1,0,0,0,0,0,1,0,0
2,4.000000,76.459948,1,0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,1,0,0,0,0,0,1,0,0
3,5.000000,76.459948,1,0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,1,0,0,0,0,0,1,0,1
4,5.000000,50.000000,0,0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0,0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,1,0,0,0,0,0,1,0,0
395,51.492308,70.000000,0,2,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,1,0,0,1,1,0,1,0,1
396,51.492308,70.000000,3,0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,1,0,0,1,1,0,0,0,0
397,51.492308,90.000000,0,0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,1,0,0,1,1,0,1,0,1


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(indep, dep, test_size=1/3, random_state=0)

In [33]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid={'solver':['lbfgs', 'liblinear', 'newton-cg','saga'],
           'penalty':['l2']}

grid=GridSearchCV(LogisticRegression(),param_grid, refit=True, verbose=3, n_jobs=-1,scoring='f1_weighted')

grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg', 'saga']},
             scoring='f1_weighted', verbose=3)

In [36]:
print(grid.best_params_)

{'penalty': 'l2', 'solver': 'lbfgs'}


In [37]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test) 

In [38]:
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_test, grid_predictions)

In [39]:
from sklearn.metrics import classification_report 
clf_report=classification_report(y_test, grid_predictions) 

In [40]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'lbfgs'}: 0.9924946382275899


In [41]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[51  0]
 [ 1 81]]


In [42]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [43]:
from sklearn.metrics import roc_auc_score
print("Roc_auc_score is:",roc_auc_score(y_test,grid.predict_proba(X_test)[:,1]))

Roc_auc_score is: 1.0


In [44]:
table=pd.DataFrame.from_dict(re)

In [45]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.037148,0.008493,0.004028,0.004934,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
1,0.011129,0.004770,0.011452,0.004078,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
2,0.020272,0.006403,0.013898,0.003646,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
3,0.028895,0.009633,0.005713,0.002519,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.981569,0.981233,0.981217,0.981217,0.981217,0.981291,0.000139,3


In [46]:
import pickle
filename="model_Log_reg_classification.sav"
pickle.dump(grid,open(filename,'wb'))

In [47]:
loaded_model=pickle.load(open("model_Log_reg_classification.sav",'rb'))      
result=loaded_model.predict([[10,70,2,0,200,50,2,130,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1]])

In [48]:
result

array([1])